In [ ]:
import pandas as pd
import numpy as np
import static_frame as sf
from datetime import datetime as dt, timedelta
from snowflake.connector import cursor
from link import model_query, aggregate_query
import xlsxwriter

# one_day_prior = dt.strftime(dt.now() - timedelta(1), '%a, %m-%d-%y')
# two_days_prior = dt.strftime(dt.now() - timedelta(2), '%a, %m-%d-%y')
error_items = [['Undefined', 0], ['TimeoutError', 1], ['BadInputError', 2], ['BadServerResponseError', 3], 
['FailedToRequestBidsError', 4], ['BidAuctionFailed', 5], ['FailedToParseResponse', 7], ['ConnectionError', 8],
['Panic', 9], ['PrebidError', 10], ['UnwrapTimeoutError', 17], ['BidPriceBelowFloor', 18], ['UnknownError', 19], 
['BidRejectedCacheID', 25], ['BidRejectedAdvertiserBlocklist', 27], ['BidRejectedIABCatBlocklist', 28], 
['BidRejectedAdomainMissing', 29], ['BidRejectedAboveMaxAdDuration', 30], ['BidRejectedAboveMinAdDuration', 31], ['NoM_FSatisfyEndpointMaxBitrate', 37],
['NoM_FSatisfyEndpointMinBitrate', 38], ['NoM_FSatisfySiteMinBitrate', 39], ['BidRejectedBrandSafetyRuleBlock', 40],
['BidPriceMissing', 44]]
error_tuple = (item[0] for item in error_items)
error_dictionary = {item[0]: item[1] for item in error_items}
error_repeat_dictionary = {item[0]: item[0] for item in error_items}
# 


def main():
    yesterday = str(dt.strftime(dt.now() - timedelta(1), '%b_%d_%Y')).upper()
    two_days_prior = str(dt.strftime(dt.now() - timedelta(2), '%b_%d_%Y')).upper()
    print("Collecting bid error data...")
    errors_by_bid_code = """
        SELECT CONCAT(PUBLISHER_ID, BIDDER_ID) AS UNIQUE_ID, PUBLISHER_ID, BIDDER_ID, BIDDER_TYPE, EVENT_BID_ERROR_CODE, SUM(CASE WHEN(CAST(DATE_TIME AS DATE) = DATEADD(day, -2, CAST(GETDATE() as DATE))) THEN EVENT_BID_ERROR ELSE 0 END) AS {}_BID_ERRORS,
        SUM(CASE WHEN(CAST(DATE_TIME AS DATE) = DATEADD(day, -1, CAST(GETDATE() AS date))) THEN EVENT_BID_ERROR ELSE 0 END) AS {}_BID_ERRORS
        FROM CONSOLE_EVENT_BUYERS_HOURLY_SUMMARY
        WHERE EVENT_BID_ERROR_CODE IN (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 17, 18, 19, 25, 27, 28, 29, 30, 31, 37, 38, 39, 40, 44)
        GROUP BY 1, 2, 3, 4, 5
    """.format(two_days_prior,yesterday)
    errors_by_bid_code = aggregate_query(errors_by_bid_code)
    errors_by_bid_code = pd.DataFrame(errors_by_bid_code)
    # errors_by_bid_code = pd.read_csv('dataset.csv')

    print("Collecting bid request data...")
    bid_requests = """
        SELECT CONCAT(PUBLISHER_ID, BIDDER_ID) AS UNIQUE_ID, 
        SUM(CASE WHEN(CAST(DATE_TIME AS DATE) = DATEADD(day, -2, CAST(GETDATE() as DATE))) THEN EVENT_BID_REQUEST ELSE 0 END) AS {}_BID_REQUESTS,
        SUM(CASE WHEN(CAST(DATE_TIME AS DATE) = DATEADD(day, -1, CAST(GETDATE() as DATE))) THEN EVENT_BID_REQUEST ELSE 0 END) AS {}_BID_REQUESTS
        FROM CONSOLE_EVENT_BUYERS_HOURLY_SUMMARY
        GROUP BY 1
    """.format(two_days_prior,yesterday)
    bid_requests = aggregate_query(bid_requests)
    bid_requests = pd.DataFrame(bid_requests)

    errors_by_bid_code = errors_by_bid_code.merge(bid_requests, on='UNIQUE_ID', how = 'left')

    print('Confirming publishers names...')
    publishers = """
        SELECT id as PUBLISHER_ID, name as PUBLISHER_NAME
        FROM publishers
    """
    publishers = model_query(publishers)
    publishers = pd.DataFrame(publishers)

    print("Matching publisher names to error codes...")
    errors_by_bid_code = errors_by_bid_code.merge(publishers, on = 'PUBLISHER_ID', how = 'left')

    print("Organizing DataFrames...")
    error_list = [error[0] for error in error_items]
    error_loop = 0
    with pd.ExcelWriter("Bid_Error_Analysis_{}-{}.xlsx".format(two_days_prior,yesterday)) as writer:
        for error in error_list:
                error_dictionary[error] = errors_by_bid_code[errors_by_bid_code['EVENT_BID_ERROR_CODE'] == int(error_dictionary.get(error))]
                error_dictionary[error]['DAY_OVER_DAY_BID_ERROR_DIFFERENCE'] = error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)] - error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)]
                error_grouping_by_id = error_dictionary[error][['PUBLISHER_ID', '{}_BID_ERRORS'.format(yesterday)]]
                error_grouping_by_id = error_grouping_by_id.groupby(['PUBLISHER_ID']).sum()
                error_grouping_by_id.rename(columns={'{}_BID_ERRORS'.format(yesterday):'YESTERDAY\'S_ERRORS'}, inplace=True)
                error_dictionary[error] = error_dictionary[error].merge(error_grouping_by_id, on = 'PUBLISHER_ID', how = 'left')
                (x, y) = (error_dictionary[error]['YESTERDAY\'S_ERRORS'], error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)])
                share = []
                j = 0
                for i in x:
                    if i == 0: 
                        share.append(0)
                    else:
                        share.append((float(y[j]) / float(x[j])) * 100)
                    j+=1
                error_dictionary[error].insert(2, 'ERROR_SHARE_ACROSS_PUB (in percent)', share)	
#                 if error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)].sum() > 0 or error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)].sum() > 0:
#                     print('Inserted total {} errors'.format(error_repeat_dictionary[error]))
#                 print('Calculating total 1-day prior (yesterday\'s) bid error percent...')            
#                 error_dictionary[error]['{}_BID_REQUESTS_AND_BID_ERRORS'.format(yesterday)] = error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)] + error_dictionary[error]['{}_BID_REQUESTS'.format(yesterday)]
                print('Calculating total 1-day prior (yesterday\'s) bid error percent...')
                (x, y) = (error_dictionary[error]['{}_BID_REQUESTS'.format(yesterday)], error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)])
                share = []
                k = 0
                for i in x:
                    if i == 0: 
                        share.append(0)
                    else:
                        share.append((float(y[k]) / float(x[k])) * 100)
                    k+=1               
                error_dictionary[error].insert(2, '{}_BID_ERROR_PERCENT'.format(yesterday), share)
                print('Calculating total 2-day prior bid requests and bid errors...')            
#                 error_dictionary[error]['{}_BID_REQUESTS_AND_BID_ERRORS'.format(two_days_prior)] = error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)] + error_dictionary[error]['{}_BID_REQUESTS'.format(two_days_prior)]
                (x, y) = (error_dictionary[error]['{}_BID_REQUESTS'.format(two_days_prior)], error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)])
                share = []
                l = 0
                for i in x:
                    if i == 0: 
                        share.append(0)
                    else:
                        share.append((float(y[l]) / float(x[l])) * 100)
                    l+=1
                print('Calculating total 2-day prior bid error percent...')                            
                error_dictionary[error].insert(2, '{}_BID_ERROR_PERCENT'.format(two_days_prior), share)
                error_dictionary[error]['+/- CHANGE IN BID ERROR %'] = error_dictionary[error]['{}_BID_ERROR_PERCENT'.format(yesterday)] - error_dictionary[error]['{}_BID_ERROR_PERCENT'.format(two_days_prior)]
                error_dictionary[error]['ERROR_SHARE_ACROSS_PUB (in percent)'] = round(error_dictionary[error]['ERROR_SHARE_ACROSS_PUB (in percent)'], 2)
                print('Calculating percent change in bid error percent...')                        
                error_dictionary[error].sort_values(by = '+/- CHANGE IN BID ERROR %', ascending = False, inplace=True)
                print('Sorting DataFrame...')            
                error_dictionary[error] = error_dictionary[error][['EVENT_BID_ERROR_CODE', 'PUBLISHER_ID', 'PUBLISHER_NAME', 'BIDDER_ID', 'BIDDER_TYPE', '{}_BID_ERRORS'.format(two_days_prior),  
                '{}_BID_REQUESTS'.format(two_days_prior), '{}_BID_ERROR_PERCENT'.format(two_days_prior), '{}_BID_ERRORS'.format(yesterday), '{}_BID_REQUESTS'.format(yesterday),  '{}_BID_ERROR_PERCENT'.format(yesterday), 'DAY_OVER_DAY_BID_ERROR_DIFFERENCE', '+/- CHANGE IN BID ERROR %', 'ERROR_SHARE_ACROSS_PUB (in percent)']]
                print('Removing empty DataFrames and rows...')
                if error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)].sum() > 0 or error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)].sum() > 0:
                    error_dictionary[error] = error_dictionary[error][(error_dictionary[error]['{}_BID_ERRORS'.format(two_days_prior)] > 0) | (error_dictionary[error]['{}_BID_ERRORS'.format(yesterday)] > 0)]
                    error_dictionary[error].to_excel(writer, sheet_name=str(error_items[error_loop][0]))
                error_loop += 1
    
# 	print('Writing DataFrames to Excel Object...')
# 		i = 0
# 		for item in error_print:               
# 			item.to_excel(writer, sheet_name=error_items[i[0]])
# 			i =+ 1
if __name__ == '__main__':
        main()